In [ ]:
from vosk import Model,KaldiRecognizer
import pyaudio
import json
import numpy as np
import resampy
import re

In [ ]:
def main():
    def PostProcess(text):
        for phrase,lingo in Lingo.items():
            pattern = r"\b" + re.escape(phrase) + r"\b"
            text = re.sub(pattern, lingo, text, flags=re.IGNORECASE)
        return text

    Lingo = {
        "to be": "dube",
        "to pay": "dube",
        "doobie": "dube",
        "dubai": "dube",
        "dubey": "dube",
        "sure jet": "surojit"
    }

    ClubMembers = ["nishant","jatin","aditya","dube","surojit"]

    WakeWordModel = Model("vosk-model-en-us-0.22-lgraph")
    CommandModel = Model("vosk-model-en-in-0.5")
    WakeWord = "hey jarvis"
    WakeWordGrammar = json.dumps([WakeWord, "[unk]"])
    WakeWordRecogniser = KaldiRecognizer(WakeWordModel, 16000, WakeWordGrammar)
    CommandRecogniser = KaldiRecognizer(CommandModel, 16000)

    CaptureRate = 48000
    VoskRate = 16000

    Listening = False

    mic = pyaudio.PyAudio()
    stream = mic.open(format=pyaudio.paInt16, channels=1, rate=CaptureRate, input=True, frames_per_buffer=8192)
    stream.start_stream()

    print("Listening for wake word...")
    while True:
        data = stream.read(4096, exception_on_overflow=False)

        NumPyAudio = np.frombuffer(data, dtype=np.int16)
        NumPyResampled = resampy.resample(NumPyAudio, CaptureRate, VoskRate)
        ResampledData = NumPyResampled.astype(np.int16).tobytes()
        
        ActiveRecogniser = CommandRecogniser if Listening else WakeWordRecogniser

        if ActiveRecogniser.AcceptWaveform(ResampledData):
            if Listening:
                ResultJSON = ActiveRecogniser.Result()
                ResultText = json.loads(ResultJSON).get("text", "")
                
                ClearLine = " " * 80 
                print(f"\r{ClearLine}\r", end="")

                if ResultText:

                    FinalText = PostProcess(ResultText)

                    print(f"User: {FinalText}")
                
                    words = FinalText.lower().split()

                    detected_members = [name for name in ClubMembers if name in words]
                    
            
                    if detected_members:
                        for member in detected_members:
                            print(f"Detected Member: {member.capitalize()}")
                
                print("\nListening for wake word...")
                Listening = False
            
            else:
                pass

        else:
            PartialJSON = ActiveRecogniser.PartialResult()
            PartialText = json.loads(PartialJSON).get("partial", "")

            if not Listening and WakeWord in PartialText:
                ClearLine = " " * 80
                print(f"\r{ClearLine}\r", end="")
                print(f"Wake word detected! Listening for a command...")
                
                Listening = True
                try:
                    ClearFrames = stream.get_read_available()
                    if ClearFrames > 0:
                        stream.read(ClearFrames, exception_on_overflow=False)
                except IOError as e:
                    print(f"Error clearing buffer: {e}")

                continue
            
            if PartialText:
                print(f"Hearing: {PartialText}\r", end="")

main()